In [98]:
# ddqn = DDQN(latent_dim, action_dim).to(device)
# optimizer = torch.optim.Adam(ddqn.parameters(), lr=learning_rate)

# for epoch in range(num_epochs):
#     for (state, action, reward, next_state, done) in replay_buffer:
#         state = vae.encoder(state).detach()
#         next_state = vae.encoder(next_state).detach()
#         q_values = ddqn(state)
#         next_q_values = ddqn(next_state)
#         target = reward + gamma * torch.max(next_q_values, dim=1)[0] * (1 - done)
#         loss = F.mse_loss(q_values.gather(1, action.unsqueeze(1)), target.unsqueeze(1))
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

import torch
import numpy as np
from torch import nn
import random
import torch.nn.functional as F
import collections
from torch.optim.lr_scheduler import StepLR
from CommunicationP3DX import CommunicationP3DX
from Agent import AgentClass
from std_srvs.srv import Empty
import rospy
import time
import shelve
import os
from datetime import datetime
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

path ='/media/xnd/7A309A87309A49D1/sia_23/10k/sim_data/'

workspace = shelve.open(path+'wsh_mean4_20231001_183307/wsh_20231003_042932.out')
# true if the key exists
vars = list(workspace.keys())
print(workspace)
print(vars)
hist_dict = workspace['hist_dict']

['Agent', 'AgentClass', 'CommunicationP3DX', 'Empty', 'Memory', 'QNetwork', 'QNetworkCNN', 'Q_1', 'StepLR', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__warningregistry__', 'actions_angular', 'actions_linear', 'checkpoint_inter', 'datetime', 'device', 'evaluate', 'filename', 'goal_zones_x', 'goal_zones_y', 'hist_dict', 'key', 'laser_scan_state_type_atual', 'main', 'max_episodes', 'max_iterations', 'my_shelf', 'n_sectors', 'path', 'pause_physics_client', 'performance', 'reset_simulation', 'select_action', 'str_hora_agr', 'str_hora_inicio_treino', 'theta_atual', 'train', 'unpause_physics_client', 'update_parameters']


In [136]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 500),
            nn.ReLU(),
            nn.Linear(500, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 500),
            nn.ReLU(),
            nn.Linear(500, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z), z

In [124]:
# creating dummy targets (float values)
episode = 9900

random_episodes =np.random.randint(2000,8000,1000)
for random_ep_idx, episode in enumerate(random_episodes):
    if random_ep_idx%100==0: print('random_ep_idx = {ep_id}'.format(ep_id =random_ep_idx))
    # shape = (727, 16)
    trasnposed_scans = np.asarray([np.asarray(scan_i) for scan_i in hist_dict['scan'][episode]]).transpose()    

    if random_ep_idx==0:
        df_episode_positions = pd.DataFrame({#'i':range(len(hist_dict['pos'][episode])),
                                            'x':[pos[0] for pos in hist_dict['pos'][episode]], 
                                            'y':[pos[1] for pos in hist_dict['pos'][episode]]})

        for scan_idx in range(0,727):
            df_episode_positions['scan_'+str(scan_idx)] = pd.to_numeric([float(scan_i)  for scan_i in trasnposed_scans[scan_idx]])

    else:
        new_episode_positions = pd.DataFrame({'x':[pos[0] for pos in hist_dict['pos'][episode]], 
                                            'y':[pos[1] for pos in hist_dict['pos'][episode]]})
        
        for scan_idx in range(0,727):
            new_episode_positions['scan_'+str(scan_idx)] = pd.to_numeric([float(scan_i)  for scan_i in trasnposed_scans[scan_idx]])

        df_episode_positions = pd.concat([df_episode_positions,new_episode_positions])
        

    df_episode_positions = df_episode_positions.replace(np.Inf, np.NaN).fillna(5)

df_episode_positions.to_pickle('dataset_1_vae.pickle')

random_ep_idx = 0
random_ep_idx = 100
random_ep_idx = 200
random_ep_idx = 300
random_ep_idx = 400
random_ep_idx = 500
random_ep_idx = 600
random_ep_idx = 700
random_ep_idx = 800
random_ep_idx = 900


In [126]:
# creating tensor from targets_df 
data = df_episode_positions.values # .to_numpy()
torch_tensor = torch.tensor(data)

# printing out result
print(torch_tensor)

tensor([[-3.6178,  2.0000,  2.0961,  ...,  5.0000,  5.0000,  5.0000],
        [-3.2115,  2.1438,  2.1437,  ...,  5.0000,  5.0000,  5.0000],
        [-2.5263,  2.2658,  3.6359,  ...,  0.7862,  0.7772,  0.7708],
        ...,
        [-8.7927,  8.5720,  1.5012,  ...,  5.0000,  5.0000,  5.0000],
        [-8.6550,  8.0715,  1.3688,  ...,  2.1308,  2.1094,  2.1062],
        [-8.1130,  7.9758,  5.0000,  ...,  2.3749,  2.3763,  2.3588]],
       dtype=torch.float64)


In [137]:
input_dim = 727+2 # 727 laser readings + 2 pos readings
latent_dim = 5
num_epochs = 5000
batch_size = 500

# Convert numpy array to PyTorch tensor
data_tensor = torch.from_numpy(data)

# Create a TensorDataset from the tensor
dataset = TensorDataset(data_tensor)

# Create a DataLoader from the TensorDataset
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr=1e-3
num_episodes = num_epochs
initial_lr = lr
final_lr = lr*1e-2
n_updates = num_episodes*0.7
lr_gamma =  (final_lr / initial_lr)**(1 / n_updates) # gamma

vae = VAE(input_dim, latent_dim).to(device)

optimizer = torch.optim.Adam(vae.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=n_updates, gamma=lr_gamma)


for epoch in range(num_epochs):
    for (inputs,) in dataloader:
        inputs = inputs.to(torch.float32).to(device)
        outputs, _ = vae(inputs)
        printloss =  F.mse_loss(outputs, inputs, reduction='sum') # 
        loss = F.binary_cross_entropy(outputs, inputs, reduction='sum')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    scheduler.step()
    if epoch%5==0:print(printloss)

tensor(1819959.2500, grad_fn=<MseLossBackward0>)
tensor(1780739.7500, grad_fn=<MseLossBackward0>)
tensor(1778248.2500, grad_fn=<MseLossBackward0>)
tensor(1758865.2500, grad_fn=<MseLossBackward0>)
tensor(1842617.2500, grad_fn=<MseLossBackward0>)
tensor(1851257.1250, grad_fn=<MseLossBackward0>)
tensor(1794730.2500, grad_fn=<MseLossBackward0>)
tensor(1854620.8750, grad_fn=<MseLossBackward0>)
tensor(1810173.5000, grad_fn=<MseLossBackward0>)
tensor(1785196.3750, grad_fn=<MseLossBackward0>)
tensor(1828045.2500, grad_fn=<MseLossBackward0>)
tensor(1775494.7500, grad_fn=<MseLossBackward0>)
tensor(1858457.1250, grad_fn=<MseLossBackward0>)
tensor(1844986.2500, grad_fn=<MseLossBackward0>)
tensor(1783885.5000, grad_fn=<MseLossBackward0>)
tensor(1770342.2500, grad_fn=<MseLossBackward0>)
tensor(1788129.8750, grad_fn=<MseLossBackward0>)
tensor(1765796.5000, grad_fn=<MseLossBackward0>)
tensor(1734397.2500, grad_fn=<MseLossBackward0>)
tensor(1809012.5000, grad_fn=<MseLossBackward0>)
tensor(1753108., gra

KeyboardInterrupt: 